In [3]:
import pandas as pd
import numpy as np
import PyPDF2
import os
import re

regiao = pd.read_excel(r'CONFIG\UF_REGIAO.xlsx')
veiculos = pd.read_excel(r'CONFIG\VEICULOS.xlsx')

def extrair_informacoes(texto):
    informacoes = {}

    data_solicitacao = re.search(r"SOLICITACAO DE MONITORAMENTO\n(\d{2}/\d{2}/\d{4})", texto)
    if data_solicitacao:
        informacoes['DATA'] = data_solicitacao.group(1)

    horario_solicitacao  = re.search(r"SOLICITACAO DE MONITORAMENTO\n\d{2}/\d{2}/\d{4} (\d{2}:\d{2}:\d{2})", texto)
    if horario_solicitacao :
        informacoes['HORA'] = horario_solicitacao.group(1)

    num_solicitacao = re.search(r"SM: #(\d+)", texto)
    if num_solicitacao:
        informacoes['SM'] = num_solicitacao.group(1)

    origem = re.search(r"ORIGEM\nLOCAL: (.+)", texto)
    if origem:
        informacoes['ORIGEM.LOCAL'] = origem.group(1)

    origem_cidade = re.search(r"ORIGEM.*?CIDADE/UF:\s+(.+?)\s+CEP:", texto, re.IGNORECASE | re.DOTALL)
    if origem_cidade:
        informacoes['ORIGEM'] = origem_cidade.group(1)

    destinos = re.findall(r"DESTINO \d+\nLOCAL: ([^\n]+) PREVISAO:", texto)
    if destinos:
        informacoes['DESTINO.INCLUÍDO.LOCAL'] = destinos

    destinos_cidades = re.findall(r"DESTINO.*?CIDADE/UF:\s+(.+?)\s+CEP:", texto, re.IGNORECASE | re.DOTALL)
    if destinos_cidades:
        informacoes['DESTINO INCLUÍDO'] = destinos_cidades

    destino_final = re.search(r"DESTINO FINAL\nLOCAL: ([^\n]+) PREVISAO:", texto)
    if destino_final:
        informacoes['DESTINO.LOCAL'] = destino_final.group(1)

    destino_final_cidade = re.search(r"DESTINO FINAL.*?CIDADE/UF:\s+(.+?)\s+CEP:", texto, re.IGNORECASE | re.DOTALL)
    if destino_final_cidade:
        informacoes['DESTINO'] = destino_final_cidade.group(1)

    rota = re.search(r"ROTA\n(.+)", texto)
    if rota:
        informacoes['ROTA'] = rota.group(1)

    cavalo = re.search(r"CAVALO:\s+(.*?)\s+CIDADE/UF:", texto, re.IGNORECASE)
    if cavalo:
        informacoes['PLACA_CAV'] = cavalo.group(1)

    carreta = re.search(r"CARRETA:\s+(.*?)\s+CIDADE/UF:", texto, re.IGNORECASE)
    if carreta:
        informacoes['PLACA_CARRETA'] = carreta.group(1)

    motorista = re.search(r"MOTORISTA / AJUDANTE\nCPF MOTORISTA: (\d+) NOME:(.+) FONE:(.+)", texto)
    if motorista:
        informacoes['CPF'] = motorista.group(1)
        informacoes['MOTORISTA'] = motorista.group(2)
        informacoes['TELEFONE'] = motorista.group(3)

    ajudante = re.search(r"CPF AJUDANTE: (\d+) NOME:(.+) FONE:(.+)", texto)
    if ajudante:
        informacoes['CPF_2'] = ajudante.group(1)
        informacoes['MOTORISTA_2'] = ajudante.group(2)
        informacoes['TELEFONE_2'] = ajudante.group(3)

    isca = re.search(r"ISCA: (\d+)", texto)
    if isca:
        informacoes['ISCA'] = isca.group(1)

    # Extrair manifesto a partir da sessão de observações até a palavra "manifesto" em maiúsculo
    observacoes = re.search(r"OBSERVACOES\n([\s\S]+?)(?=\nMANIFESTO)", texto, re.IGNORECASE)
    if observacoes:
        manifestos = re.findall(r"Manifesto ([^\n]+)", observacoes.group(1))
        if manifestos:
            informacoes['MANIFESTO'] = manifestos

    # Extrair manifesto a partir da sessão "MANIFESTO" até a sessão "DADOS COMPLEMENTARES"
    manifesto_complementares = re.findall(r"MANIFESTO([\s\S]+?)DADOS COMPLEMENTARES", texto, re.DOTALL)
    if manifesto_complementares:
        informacoes['MANIFESTO'] = manifesto_complementares[0].split('\n')
                            
    observacoes = re.findall(r"OBSERVACOES\n(.+?)(?=MANIFESTO\s)", texto, re.DOTALL)
    if observacoes:
        informacoes['OBSERVAÇÃO'] = observacoes[0].split('\n')

    valor_viagem = re.search(r"VALOR DA VIAGEM: R\$ (\d+(?:\.\d+)?)", texto)
    if valor_viagem:
        informacoes['VALOR R$'] = valor_viagem.group(1)

    return informacoes


def extrair_informacoes_pdf(caminho_pdf):
    informacoes = {}

    with open(caminho_pdf, 'rb') as arquivo_pdf:
        reader = PyPDF2.PdfReader(arquivo_pdf)

        texto_completo = ""
        for pagina in reader.pages:
            texto_completo += pagina.extract_text()

        informacoes = extrair_informacoes(texto_completo)

    return informacoes

def determinar_tipo_rota(origem, destino,rota):
    origem_lower = origem.lower()
    destino_lower = destino.lower()
    rota_lower = rota.lower()

    if 'matriz' in origem_lower and 'matriz' in destino_lower:
        return 'COLETA'
    elif rota_lower=='sim' and origem_lower==destino_lower:
        return 'COLETA'
    elif 'matriz' in origem_lower or 'matriz' in destino_lower:
        return 'PRIMARIA'
    else:
        return 'SECUNDARIA'

def determinar_rota_unificada(tam):
    if tam > 1:
        return 'NÃO'
    elif tam == 0:
        return 'NÃO'
    else:
        return 'SIM'

def determinar_trecho(origem, destino):
    origem_lower = origem.lower()
    destino_lower = destino.lower()

    if 'matriz' in origem_lower and 'matriz' in destino_lower:
        return 'IN'
    elif 'matriz' in destino_lower:
        return 'OUT'
    else:
        return 'IN'
    
def separar_lista(coluna):
    resultado_formatado = ' / '.join(coluna)
    return resultado_formatado

#Configurações iniciais
caminho_pdf = r'C:\Users\karina.teotonio\Desktop\Karina Cristina\SMs Script\SM'
lista_pdf = os.listdir(caminho_pdf)
pdfVazio = pd.DataFrame()

for file in lista_pdf:
    informacoes_pdf = extrair_informacoes_pdf(fr'{caminho_pdf}/{file}')

    #Transformando em DF
    pdfExcel = pd.json_normalize(informacoes_pdf)

    #Removendo colunas desnecessárias
    pdfExcel = pdfExcel.drop(columns=['TELEFONE'])

    if 'DESTINO.INCLUÍDO.LOCAL' not in pdfExcel.columns.tolist():
        pdfExcel['TAM'] = 0
    else:
        pdfExcel['TAM'] = pdfExcel['DESTINO.INCLUÍDO.LOCAL'].apply(len)

    pdfExcel['ROTA UNIFICADA?'] = pdfExcel.apply(lambda row: determinar_rota_unificada(row['TAM']), axis=1)
    
    pdfExcel['TIPO_ROTA'] = pdfExcel.apply(lambda row: determinar_tipo_rota(row['ORIGEM.LOCAL'], row['DESTINO.LOCAL'],row['ROTA UNIFICADA?']), axis=1)

    #Adicionando a região
    pdfExcel['UF'] = pdfExcel['DESTINO'].str[-2:]
    pdfExcel = pdfExcel.merge(right=regiao, on='UF').drop(columns=['UF'])

    #Adicionando a empresa
    #pdfExcel = pdfExcel.merge(right=veiculos, on='PLACA_CAV')

    if 'DESTINO.INCLUÍDO.LOCAL' in pdfExcel.columns.tolist():
        pdfExcel['DESTINO INCLUÍDO'] = np.where(pdfExcel['ROTA UNIFICADA?']=='NÃO','',pdfExcel['DESTINO INCLUÍDO'][0][0])
        #pdfExcel['TIPO_OPERACIONAL'] = np.where(pdfExcel['ROTA UNIFICADA?']=='NÃO','',pdfExcel['DESTINO.INCLUÍDO.LOCAL'][0][0])
    else:
        pdfExcel['DESTINO INCLUÍDO'] = ''
    #pdfExcel['ORIGEM'] = np.where(pdfExcel['ROTA UNIFICADA?']=='NÃO',pdfExcel['ORIGEM'],pdfExcel['ORIGEM.LOCAL'])
    #pdfExcel['DESTINO'] = np.where(pdfExcel['ROTA UNIFICADA?']=='NÃO',pdfExcel['DESTINO'],pdfExcel['DESTINO.LOCAL'])

    pdfExcel['TRECHO'] = pdfExcel.apply(lambda row: determinar_trecho(row['ORIGEM.LOCAL'], row['DESTINO.LOCAL']), axis=1)

    #Criando colunas para o DF
    verificacao_de_colunas = ['DATA','HORA','SM','VALOR R$','ORIGEM','ROTA UNIFICADA?','DESTINO INCLUIDO','DESTINO','ROTA','REGIAO','TIPO_ROTA','TIPO_OPERACIONAL','PLACA_CAV','PLACA_CARRETA','EMPRESA','MOTORISTA','CPF','MOTORISTA_2','CPF_2','ISCA','MANIFESTO','OBSERVAÇÃO','TRECHO','OCORRÊNCIA','LOCAL DA OCORRÊNCIA','UF do LOCAL DA OCORRÊNCIA','CATEGORIA','PRIORIDADE','IMAGEM','OCORRÊNCIA_2','LOCAL DA OCORRÊNCIA_2','UF do LOCAL DA OCORRÊNCIA_2','CATEGORIA_2','PRIORIDADE_2','IMAGEM_2']
    for i in verificacao_de_colunas:
        if i not in pdfExcel.columns.tolist():
            pdfExcel[i] = ''

    #Validação de colunas, para a criação
    if 'MOTORISTA_2' not in pdfExcel.columns.tolist():
        pdfExcel[['MOTORISTA_2','CPF_2']] = ''

    # Campo de ocorrências sendo preenchido
    pdfExcel['OCORRÊNCIA'] = 'NÃO'

    pdfExcel['MANIFESTO'] = pdfExcel['MANIFESTO'].apply(separar_lista)
    pdfExcel['OBSERVAÇÃO'] = pdfExcel['OBSERVAÇÃO'].apply(separar_lista)

    pdfExcel['SM'] = pd.to_numeric(pdfExcel['SM'])
    pdfExcel['CPF'] = pd.to_numeric(pdfExcel['CPF'])
    pdfExcel['VALOR R$'] = pdfExcel['VALOR R$'].str.replace('.','',regex=False)
    pdfExcel['VALOR R$'] = pdfExcel['VALOR R$'].astype(int)

    # Verificar e substituir o valor para nulo usando expressão regular
    texto_alvo = 'RODOBENS GERENCIAMENTO DE RISCO'
    pdfExcel = pdfExcel.replace(re.compile(r'\b' + re.escape(texto_alvo) + r'\b'), None, regex=True)

    #Reordenando as colunas do DF
    pdfExcel = pdfExcel[['DATA','HORA','SM','VALOR R$','ORIGEM','ROTA UNIFICADA?','DESTINO INCLUÍDO','DESTINO','ROTA','REGIAO','TIPO_ROTA','TIPO_OPERACIONAL','PLACA_CAV','PLACA_CARRETA','EMPRESA','MOTORISTA','CPF','MOTORISTA_2','CPF_2','ISCA','MANIFESTO','OBSERVAÇÃO','TRECHO','OCORRÊNCIA','LOCAL DA OCORRÊNCIA','UF do LOCAL DA OCORRÊNCIA','CATEGORIA','PRIORIDADE','IMAGEM','OCORRÊNCIA_2','LOCAL DA OCORRÊNCIA_2','UF do LOCAL DA OCORRÊNCIA_2','CATEGORIA_2','PRIORIDADE_2','IMAGEM_2']]

    pdfVazio = pd.concat([pdfVazio,pdfExcel])
#Exportando o arquivo para Excel
pdfVazio.to_excel('SMs.xlsx',index=False)